### Coding text fragments
This notebook guides you through the process of coding trade journals. These steps are as follows:

1. Determine relevancy of trade journal for the study.
2. Split journal into smaller chunks.
3. Determine relevancy of each chunk.
4. Code relevant chunks based on coding scheme.

In this specific case, the LLM is asked to code trade journals with the aim of mapping the socio-technical configurations in the Dutch energy system.

### Install the necessary packages

In [ ]:
import pandas as pd
from langchain.chat_models import ChatOpenAI # Replace with package required for your desired model
from dotenv import load_dotenv
import os
from pydantic import BaseModel, Field 
from langchain_openai import ChatOpenAI # Replace with package required for your desired model
from google.cloud import bigquery
from tqdm import tqdm
import numpy as np
from llama_index.core.node_parser import SemanticSplitterNodeParser, SentenceSplitter
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Document
import tiktoken
from pathlib import Path

### Initialize model
Here, gpt-4o-mini is used. However, any model that is compatible with LangChain's with_structured_output can be used.

In [ ]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0, disable_streaming=True) # Specify api key in .env file

### Load data

In [ ]:
# Load api key
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "key.json" # Add Google Cloud credentials to key.json file

# Make BigQuery client
client = bigquery.Client()

# Retrieve data and transform to dataframe
table_id = 'content.source' # Replace with table name (projectname.tablename)
df = client.list_rows(table_id).to_dataframe()

print(df.head())
print(f"\nKolommen:")
print(df.columns.tolist())

### Basemodels
The following Pydantic basemodels are used to ensure that data fits a certain schema. The prompts are also written in these basemodels.

Basemodel for determining relevancy of a text fragment:

In [ ]:
# Adapt based on your research goals.
class ContentData(BaseModel):
  """
  Je krijgt een dataset met daarin artikelen en stukken tekst uit artikelen van oude Nederlandse tijdschriften. Ik wil je dat een artikel/stuk uit een artikel bewaard als het voldoet aan het volgende:

  Ik ben op zoek naar tekstfragmenten die relevante socio-technische configuraties beschrijven. Dit kan op twee manieren:
  1. Discursieve focus: Fragmenten waarin actoren (zoals bedrijven, beleidsmakers of experts) uitspraken doen over de geschiktheid van bepaalde technologieën om problemen op te lossen, de regels rond het gebruik ervan, of waarin ze combinaties van technologische en institutionele elementen evalueren. Bijvoorbeeld: meningen over nieuwe technologieën, discussies over regelgeving, of uitspraken over de impact van technologie op de sector.
  2. Substantiële focus: Fragmenten die concrete activiteiten of institutionele veranderingen beschrijven waarin technologieën en sociale elementen samenkomen. Dit kunnen berichten zijn over grote investeringen, promotieactiviteiten, het openen van nieuwe markten, de oprichting van onderzoekscentra, of andere gebeurtenissen waarin technologie en instituties samen een rol spelen.
  Selecteer alleen die artikelen/stukken tekst met fragmenten waarin **expliciet** een combinatie van technologische en sociale/institutionele elementen wordt genoemd. Vermijd algemene beschrijvingen van technologieën zonder verwijzing naar sociale of institutionele aspecten.

  Verder is het belangrijk dat het tekstfragment **direct** betrekking heeft op het **Nederlandse energiesysteem**. Dit kan zijn energie productie, transmissie, conversie, opslag of gebruik. Een trein gebruikt om kolen te vervoeren is hierbij wel relevant, maar een trein op zichzelf niet.

  1. Technologieën: Technologie is het geheel van materiële en immateriële entiteiten, vaardigheden en activiteiten waarmee mensen mentale en fysieke inspanning toepassen om hun omgeving te veranderen, input omzetten in output en de toegang tot hulpbronnen verbeteren. Het is een essentieel onderdeel van het sociotechnische en socioculturele landschap en belichaamt kennis, rationaliteit en ideologie, terwijl het zowel bedoelde als onbedoelde gevolgen met zich meebrengt.
  2. Markten: Een markt is een dynamisch en gedecentraliseerd systeem van vraag en aanbod, waarbij productieve factoren worden toegewezen via prijssignalen. Het wordt gedreven door competitie, de maximalisatie van consumentennut en het streven naar winst door producenten, wat leidt tot structurele overgangen in productie, investeringen en consumptie. Innovatie en ondernemerschap herschikken voortdurend marktstructuren door creatieve destructie, terwijl vrijwillige uitwisselingen, geleid door eigenbelang, bijdragen aan algehele efficiëntie en welzijn. 
  3. Instituties: Een institutie is een systeem van formele en informele regels die menselijk gedrag structureren, interacties vormgeven en regelmatige patronen van praktijk creëren. Deze regels, die voortkomen uit collectieve actie en interactie, functioneren als beperkingen en mogelijkheden voor gedrag, waardoor complexiteit en coördinatiekosten worden verminderd. Instellingen kunnen expliciet of impliciet, descriptief of prescriptief zijn, en omvatten formele (zoals wetten, regelgeving en voorschriften) en informele regels (zoals overtuigingen en waarden).
  4. Actoren: Alle actoren die invloed hebben op het energiesysteem, zoals bedrijven, overheden, netbeheerders, en andere stakeholders die betrokken zijn bij de energietransitie. Wanneer je dit niet kan vinden, hoef je hier niet voor te coderen.

  **Wel** relevant:
  - Tekstfragmenten waarin actoren iets zeggen over technologieën, instituties en/of markten gerelateerd aan het energiesysteem.
  - Tekstfragmenten over innovaties gerelateerd aan het energiesysteem.
  - Tekstfragmenten over aanbevelingen gerelateerd aan het energiesysteem.
  - Een fragment is alleen relevant als je expliciet kunt aanwijzen welke Nederlandse actor, technologie, markt of institutie wordt besproken. Als deze niet te identificeren zijn, is het fragment niet relevant.
  
  **Niet** relevant:
  - Tekstfragmenten gerelateerd aan abonnement of tijdschriftinformatie.
  - Tekstfragmenten over het *buitenland*. Alles wat over het buitenland (bijv. Engeland of Duitsland) gaat is niet relevant. Classificeer een tekstfragment waarin bijvoorbeeld London of een andere buitenlandse stad wordt genoemd als **niet relevant**.
  - Tekstfragmenten die reclame bevatten.

  """
  titel: str = Field(..., description = "De volledig titel van het tijdschrift.")
  relevant: bool = Field(..., description = "Geeft aan of een tekstfragment relevant is.")

Basemodel for coding relevant text fragments:

In [ ]:
# Adapt based on your research goals
class textCoder(BaseModel):
  """ 
  Voor een onderzoek naar het Nederlandse energiesysteem moet je stukken tekst uit oude vaktijdschriften coderen.

  Stap 1: Bekijk het stuk tekst.

  Stap 2: Codeer het stuk tekst met toepasselijke codes.
  Let op:
  - Gebruik het gegeven codeerschema om te coderen.
  - Je mag het codeerschema veranderen op level 4 maar geen codes toevoegen op level 5.
  - Doe hier geen aannames. Pas alleen codes toe als er expliciet iets in de tekst staat wat het toepassen van een code rechtvaardigd. 
  - Wanneer een formele institutie bediscussieerd wordt codeer je voor een formele institutie en een informele institutie.
  - Het gebruik van water of de benoeming van waterleidingen betekent niet gelijk dat er gebruik wordt gemaakt van water als energiebron. Codeer alleen voor waterkracht wanneer dit explicitiet genoemd wordt.
  - Bij ergens in geloven (of juist niet), argumenten voor of tegen iets, discussies en aanbevelingen codeer je ook voor informele institutie omdat dit aangeeft wat mensen hun overtuigingen zijn. Ook het toetsen van bijvoorbeeld een wet door proeven te doen kan je zien als een overtuiging.
  - Benoemingen van mensen waarbij het onduidelijk is wie zij zijn codeer je als de vereniging waar zij lid van zijn (bijv. in het tijdschrift Het Gas van de Vereeniging van Gasfabrikanten in Nederland) of als diverse.
  - Gebruik de code 'Gemeente onbekend' alleen als de specifieke gemeente niet duidelijk wordt uit de tekst. Anders kan je coderen met een gemeente uit het codeerschema of daar zelf een aan toevoegen.
  
  Stap 3: Ga na of je alle codes die toepasselijk zijn hebt toegevoegd.

  Stap 4: Mocht je codes vergeten zijn, voeg deze dan nog toe.
  Stel hierbij de volgende controle vragen:
  - Wanneer er een of meerdere actoren benoemd worden, is hiervoor gecodeerd?
  - Wanneer er een of meerdere technologieën genoemd worden, is hiervoor gecodeerd?
  - Waneer er een of meerdere markten genoemd worden, is hiervoor gecodeerd?
  - Wanneer er een of meerdere instituties genoemd worden, is hiervoor gecodeerd?
  - Wanneer er een locatie genoemd wordt, is hiervoor gecodeerd?
  - Wanneer er een jaartal genoemd wordt, is hiervoor gecodeerd?

  Stap 5: Als er geen jaartal uit de tekst te halen is, codeer dit dan vanuit de titel van het tijdschrift.

  Stap 6: Geef een uitleg over de gekozen codes.
  
  Let op:
  - Benoem in je uitleg alle gegeven codes.

  Voorbeeld:
    Stuk tekst: "De eerste kamer heeft recent en wet geïntroduceerd die bedrijven aanmoedigt om te investeren in zonne-energie. Dit heeft geleid tot een verhoogde vraag naar zonnepanelen."
    Toegewezen codes:
    - Actor > Overheid > Parlement > Eerste kamer
    - Concept > Institutie > Formeel > Economische institutie
    - Concept > Technologie > Zon > Productie
    - Concept > Markt > Toepassing > Industrie
  """
  titel: str = Field(..., description = "De volledig titel van het tijdschrift.")
  text: str = Field(..., description="Een relevant stuk tekst.")
  codes: list[str] = Field(..., description="Een lijst met een of meerdere codes per tekstfragment.")
  uitleg: str = Field(..., description="Uitleg over waarom de codes passen bij het tekstfragment.")

### Provide coding scheme
A preliminary coding scheme is given to guide the model when coding the text fragments. In this notebook, the model has the freedom to make minor changes to the coding scheme.

In [ ]:
# Load the coding scheme
# In this case,  codeerschema.csv is an example of what it could look like

codeerschema = pd.read_csv("codeerschema.csv", encoding="utf-8-sig", on_bad_lines="skip", sep=";")
print(codeerschema)
code_prompt = "Hier is het codeerschema dat je moet gebruiken om de tekst te analyseren:\n\n"
for _, row in codeerschema.iterrows():
    code_prompt += f"- **{row['Code level 1']} > {row['Code level 2']} > {row['Code level 3']} > {row['Code level 4']} **:" \
                   f"(Bijvoorbeeld: {row['Voorbeeld 1']}, {row['Voorbeeld 2']}, {row['Voorbeeld 3']}, {row['Voorbeeld 4']}, {row['Voorbeeld 5']})\n"

### Functions
Below all functions are given to execute the three steps of the coding process. The functions use models that are wrapped with the different basemodels defined earlier.

Functions for chunking journal content:

In [ ]:
# This function counts the amount of tokens in a chunk
# This is used in the split_content() to determine whether further chunking is necessary
def count_tokens(text):
    """Counts the number of tokens in a text using tiktoken."""
    encoding = tiktoken.encoding_for_model("gpt-4o-mini")
    return len(encoding.encode(text))

In [ ]:
# This function splits the content of journals into smaller nodes based on semantic and sentence-level splitting.
# It first performs semantic splitting to break the text into meaningful chunks with the aim of identifying articles.
# Then chunk size is checked.
# If a chunk is too large, it further splits it into smaller pieces using a sentence splitter.
# The function returns a list of nodes, each representing a smaller chunk of the original content.

def split_content(row):
    embed_model = OpenAIEmbedding(model="text-embedding-3-small")
    node_parser = SemanticSplitterNodeParser(
        buffer_size=1, breakpoint_percentile_threshold=90, embed_model=embed_model
    )
    sentence_splitter = SentenceSplitter(chunk_size= 400, chunk_overlap= 100) 

    nodes = []
    fragments = [] 

    text = str(row['content'])
    titel = str(row['full_title'])

    # Step 1: Semantic splitting
    doc = Document(text=text, metadata={"title": titel})
    split_nodes = node_parser.get_nodes_from_documents([doc])

    for node in split_nodes:
        node.metadata["title"] = titel

        # Step 2: Check if the node > 500 tokens
        num_tokens = count_tokens(node.text)
        if num_tokens > 500:
            smaller_nodes = sentence_splitter.get_nodes_from_documents([Document(text=node.text)])

            for small_node in smaller_nodes:
                small_node.metadata["title"] = titel
                nodes.append(small_node)
                fragments.append({'title': titel, 'text': small_node.text}) 
        else:
            nodes.append(node)
            fragments.append({'title': titel, 'text': node.text}) 

    ts_fragments = pd.DataFrame(fragments)

    return nodes, ts_fragments

Function for determining relevancy of text fragments:

In [ ]:
# This function filters relevant text fragments based on their content for the energy system.
# It iterates through each row in the DataFrame, constructs a prompt with the fragment's title and text, and invokes a language model to determine if the fragment is relevant. 
# Relevant fragments are added to a list, and errors during processing are logged.
relevant_llm = model.with_structured_output(ContentData, method="json_schema")

def filter_relevant_fragments(df):
    relevant_fragments = [] 
    errors = [] 
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing fragments"):
        input_text = f"""
        Bepaal of een tekstfragment relevant is voor het energiesysteem.
        
        Tijdschrift: 
        {row["title"]}
        
        Tekst:
        {row["text"]}
        """
        try:
            result = relevant_llm.invoke(input_text)

            if result.relevant:
                relevant_fragments.append(row)
        
        except Exception as e:
            print(f"Error processing {row['title']}: {e}")
            errors.append({"tekstfragment": row["text"], "error_message": str(e)})

    return pd.DataFrame(relevant_fragments) if relevant_fragments else None, errors

Function for coding the text:

In [ ]:
code_llm = model.with_structured_output(textCoder, method="json_schema")

# This function applies text coding to journal fragments based on a given coding schema.
# It iterates through each row in the DataFrame, constructing a prompt with the journal title and text fragment.
# The function then invokes a language model to generate structured codes and explanations.
# The results, including the coded text, assigned codes, and explanations, are stored in a list.
# Any errors encountered during processing are logged.
# The function returns a DataFrame of coded results or None if no results are found, along with any errors.
def code_text(df):
  results = []
  errors = []

  for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing journals"):
    input_text = f""" 
    Codeer tekstfragmenten, rekening houdend met de context van het fragment die in de tekst te vinden is.

    Tijdschrift titel:
    {row["title"]}
    
    Tekstfragment:
    {row["text"]}

    Gebruik dit codeerschema:
    {code_prompt}

    """

    try:
      result = code_llm.invoke(input_text, timeout=120)

      results.append({
          "full_title": row['title'], 
          "tekstfragmenten": result.text,
          "codes": result.codes,  
          "uitleg": result.uitleg
      })

    except Exception as e:
      print(f"Error processing {row['text']}: {e}")
      errors.append({"tekstfragment": row["text"], "error_message": str(e)})
  
  return pd.DataFrame(results) if results else None, errors

Function for coding the text in batches:

In [ ]:
# This function processes a DataFrame in batches to apply text coding efficiently.
# It splits the DataFrame into smaller batches (default size = 10) and iterates over them.
# For each batch, it calls the `code_text` function to process the text fragments.
# If coding is successful, the results are stored; any errors encountered are logged.
# Finally, it combines all processed batches into a single DataFrame and returns it along with any errors.
def process_in_batches(df, batch_size=10): # Adjust batch_size as desired
  results = []
  errors = []

  num_batches = int(np.ceil(len(df) / batch_size))  # Calculate number of batches for progress bar

  for batch_num in tqdm(range(num_batches), desc="Processing batches", unit="batch"):
      batch = df.iloc[batch_num * batch_size: (batch_num + 1) * batch_size]

      # Code fragments per batch
      try:
          batch_results, code_errors = code_text(batch)
          if batch_results is not None:
              results.append(batch_results) 
        
          errors.extend(code_errors)
          
      except Exception as e:
          print(f"Error processing batch {batch_num}: {e}")
          errors.append({"batch_num": batch_num, "error_message": str(e)})

  # Combine results 
  return pd.concat(results, ignore_index=True) if results else None, errors

### Execution
There are two ways to execute the process of coding. The first approach is great for testing on a smaller sample. The second approach uses OpenAI's Batch API. This approach is recommended when working with large amounts of data. Code for both approaches is given below.

#### Approach one
The following code executes the process of coding fragments step by step.


Step 1: split journal content into smaller chunks

In [ ]:
fragments = split_content(df)

In [ ]:
# Create DataFrame with all fragments and corresponding journal titles
fragments = pd.DataFrame({
    'title': [node.metadata['title'] for node in fragments],
    'text': [node.text for node in fragments] 
})
print(fragments.head())
fragments.to_html('ts_fragments.html')

Step 2: extract relevant fragments

In [ ]:
# Read data
fragments = pd.read_html('fragments.html')
fragments = fragments[0]

# Extract relevant fragments
if fragments is not None:
    print(f"Aantal fragmenten: {len(fragments)}")

    relevant_fragments, fragment_errors = filter_relevant_fragments(fragments)
    relevant_fragments.to_html('relevant_fragments.html')

    # If there are any errors, save to a csv
    if fragment_errors:
        fragment_errors.to_csv('fragment_errors.csv')
        print(f"Aantal errors: {len(fragment_errors)}")

Step 3: code fragments

In [ ]:
# Read data
fragments = pd.read_html('relevant_fragments.html')
fragments = fragments[0]

# Function to code fragments in batches and save results
final, final_errors = process_in_batches(fragments) 
final.to_html('ts_final.html')

# If there are any errors, save to a csv
if final_errors:
  final_errors.to_csv('final_errors.csv')
  print(f"Aantal errors: {len(final_errors)}")

#### Approach two: OpenAI Batch API
OpenAI's Batch API is useful when you have a lot of requests for which you do not need immediate response. You can send a jsonl file with all requests which OpenAI executes within 24 hours (often sooner) for 50% of the cost. See https://platform.openai.com/docs/guides/batch for more information.

In [ ]:
from openai.lib._pydantic import to_strict_json_schema
import json

Step 1: split journal content

In [ ]:
def split_content(row):
    embed_model = OpenAIEmbedding(model="text-embedding-3-small")
    node_parser = SemanticSplitterNodeParser(
        buffer_size=1, breakpoint_percentile_threshold=90, embed_model=embed_model
    )
    sentence_splitter = SentenceSplitter(chunk_size= 400, chunk_overlap= 100) 

    nodes = []
    fragments = [] 

    text = str(row['content'])
    titel = str(row['full_title'])

    # Step 1: Semantic splitting
    doc = Document(text=text, metadata={"title": titel})
    split_nodes = node_parser.get_nodes_from_documents([doc])

    for node in split_nodes:
        node.metadata["title"] = titel

        # Step 2: Check if the node > 500 tokens
        num_tokens = count_tokens(node.text)
        if num_tokens > 500:
            smaller_nodes = sentence_splitter.get_nodes_from_documents([Document(text=node.text)])

            for small_node in smaller_nodes:
                small_node.metadata["title"] = titel
                nodes.append(small_node)
                fragments.append({'title': titel, 'text': small_node.text}) 
        else:
            nodes.append(node)
            fragments.append({'title': titel, 'text': node.text}) 

    ts_fragments = pd.DataFrame(fragments)

    return nodes, ts_fragments

In [ ]:
# This script processes a DataFrame of journal entries,
# splits each journal into smaller content fragments, saves each set of fragments as a separate CSV file, 
# and finally combines all fragments into a single CSV file.

all_fragments = []  
output_dir = "docs"
df =df.iloc[366:]

for _, row in tqdm(df.iterrows(), total=len(df), desc="Processing journals"):

    nodes, ts_fragments = split_content(row)
    ts_fragments.to_csv(os.path.join(output_dir, f"journal{_}.csv"))
    all_fragments.append(ts_fragments)  

final_fragments = pd.concat(all_fragments, ignore_index=True)

final_fragments.to_csv('final_fragments.csv')

Step 2: extract relevant fragments


Step 2.1: set up requests

In [ ]:
# Add a new column as custom_id to each row of the DataFrame to match the requests to and responses from OpenAI
df = pd.read_csv('final_fragments.csv')
df['custom_id'] = range(len(df))
df.to_csv('final_fragments.csv')

In [ ]:
# Transform basemodel into strict json schema
schema = to_strict_json_schema(ContentData)
print(schema)

In [ ]:
# This script processes the first row of a DataFrame and formats it into a structured API request for OpenAI's chat completion endpoint. 
# It constructs a task with a unique ID, sends the journal's title and content as input, and requests a relevance check using a predefined JSON schema. 
# The task is then added to a list for batch processing.

# Files may be 200MB max. It might therefore be necessary to split the data in several batches.

df = pd.read_csv('final_fragments.csv')
tasks = []

for index, row in df.iterrows():

    title = row['full_title']
    description = row['text']
    
    task = {
        "custom_id": f"task-{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini",
            "temperature": 0,
            "response_format": {
                "type": "json_schema",
                "json_schema": {
                    "name": "RelevanceCheck",
                    "schema": schema
                }
            },
            "messages": [
                {
                    "role": "user",
                    "content": f"Is dit tekstfragment relevant? Dit is de titel: {title} en de inhoud: {description}"
                }
            ],
        }
    }
    
    tasks.append(task)

In [ ]:
# Save all requests in a jsonl file
# This file can then be uploaded to https://platform.openai.com/batches to start the process
jsonl_filename = "requests_step2.jsonl"
with open(jsonl_filename, "w", encoding="utf-8") as f:
    for task in tasks:
        f.write(json.dumps(task) + "\n")

print(f"Batch bestand opgeslagen als {jsonl_filename}")

Step 2.2: process response

In [ ]:
# The response file can be downloaded from https://platform.openai.com/batches
# This function extracts the relevant information of each row (custom_id, title, and relevancy (true/false))

batch_data = []

responses = Path('responses').glob('*.jsonl')

for response in responses:
# JSONL-bestand openen en verwerken
    with open(response, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line)  # Elke regel als JSON-object laden
            
            # Extract custom_id, titel en relevantie
            custom_id = data["custom_id"]
            content_str = data["response"]["body"]["choices"][0]["message"]["content"]
            try:
                content_dict = json.loads(content_str)  # JSON-string omzetten naar dictionary
            except json.JSONDecodeError as e:
                print(response.name)
                print(custom_id)
                print(repr(content_str))

            titel = content_dict["titel"]
            relevant = content_dict["relevant"]

            # Toevoegen aan lijst als dictionary
            batch_data.append({"custom_id": custom_id, "relevant": relevant})

# Lijst omzetten naar DataFrame
df = pd.DataFrame(batch_data)

# DataFrame opslaan als CSV (optioneel)
df.to_csv("output.csv", index=False)

In [ ]:
# Responses then need to be matched to their corresponding requests
df = pd.read_csv("df.csv")
df['custom_id'] = df['custom_id'].astype(str)
df['custom_id'] = df['custom_id'].astype(str)
df_matched = df.merge(df, how="left", on="custom_id")

In [ ]:
# Drop unnecessary columns and save relevant rows (=chuncks) for the next step
df_matched = df_matched.drop(columns=["Unnamed: 0"])
step2_output = df_matched
step3_input = df_matched[df_matched['relevant']]
step3_input.to_csv('step3_input.csv')

# To check difference between step 2 output (all chunks) and step 3 input (relevant chunks)
print(len(step2_output))
print(len(step3_input))

Step 3: code

Step 3.1: set up requests

In [ ]:
# Add a new column as custom_id to each row of the DataFrame to match the requests to and responses from OpenAI
step3_input['custom_id'] = range(len(step3_input))
step3_input.to_csv('step3_input.csv')

In [ ]:
# Transform basemodel into strict json schema
schema = to_strict_json_schema(textCoder)
print(schema)

In [ ]:
# This script processes the first row of a DataFrame and formats it into a structured API request for OpenAI's chat completion endpoint. 
# It constructs a task with a unique ID, sends the journal's title and content as well as the coding scheme as input, and requests coding based on the coding scheme and JSON schema.
# The task is then added to a list for batch processing.

# Files may be 200MB max. It might therefore be necessary to split the data in several batches.

step3_input = pd.read_csv("step3_input.csv")
tasks = []

for index, row in step3_input.iterrows():

    titel = row["title"]
    fragment = row['text']
    schema = code_prompt
    
    task = {
        "custom_id": f"{index}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o-mini",
            "temperature": 0,
            "response_format": {
                "type": "json_schema",
                "json_schema": {
                    "name": "textCoder",
                    "schema": schema
                }
            },
            "messages": [
                {
                    "role": "user",
                    "content": f"Codeer de tekstfragmenten, rekening houdend met de context van het fragment die in de tekst te vinden is. Dit is de titel: {titel}. Dit is het tekstfragment: {fragment}. Dit is het codeerschema: {schema}."
                }
            ],
        }
    }
    
    tasks.append(task)

In [ ]:
jsonl_filename = "requests_step3.jsonl"
with open(jsonl_filename, "w", encoding="utf-8") as f:
    for task in tasks:
        f.write(json.dumps(task) + "\n")

print(f"Batch bestand opgeslagen als {jsonl_filename}")

Step 3.2: process responses

In [ ]:
# This script reads multiple JSONL files from the 'responses' folder,
# parses the content of each line, extracts relevant fields like 'custom_id', 'text', 'codes', and 'uitleg',
# and appends them to a list called 'batch_data'. 
# It also handles and reports errors if parsing fails.

batch_data = []

responses = Path('responses').glob('*.jsonl')

for response in responses:
# JSONL-bestand openen en verwerken
    with open(response, "r") as f:
         for line in f:
            try:
                data = json.loads(line)
            except:
                print(line)
                raise
            try:
                custom_id = data["custom_id"]
                content = textCoder.model_validate_json(data["response"]["body"]["choices"][0]["message"]["content"])

                text = content.text
                codes = content.codes
                if codes:
                    codes = list(set(codes))
                uitleg = content.uitleg
                batch_data.append({
                    "custom_id": custom_id,
                    "text": text,
                    "codes": codes,
                    "uitleg": uitleg
                })

            except:
                print(response)
                print(data["response"]["body"]["choices"][0]["message"]["content"])
                raise

In [ ]:
# Convert the collected batch_data list into a pandas DataFrame
df_codes = pd.DataFrame(batch_data)

In [ ]:
# Match the requests and response rows
# Save data to csv
df_codes['custom_id'] = df_codes['custom_id'].astype(str)
step3_input['custom_id'] = step3_input['custom_id'].astype(str)
df_matched = df_codes.merge(step3_input, how = "inner", on = "custom_id")
df_matched.to_csv('coded_fragments.csv')

The coded_fragments csv can now be used for further analysis.